<a href="https://colab.research.google.com/github/nebilarega/gesture_recog/blob/main/gesture_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data fetch and preprocessing

In [ ]:
import sys
import tarfile
import zipfile
from six.moves import urllib
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Creating path to extract file and store

In [ ]:
path = '/content/drive/MyDrive/gesture_dataset'
os.makedirs(path, exist_ok=True)
zip_path = '/content/drive/MyDrive/archive_1.zip'
gesture_zip = zipfile.ZipFile(zip_path)
gesture_zip.extractall(path=path)
gesture_zip.close()
os.remove(zip_path)

### Get the file directories and randomize the dirs

In [ ]:
real_path = os.path.join(path, 'leapGestRecog')
directories = [directories for directories in sorted(os.listdir(real_path))]

In [ ]:
data_dir = [[],[],[],[],[],[],[],[],[],[]]
for i in range(len(directories)):
  _dir = [_dir for _dir in sorted(os.listdir(os.path.join(real_path, directories[i])))]
  for j in range(len(_dir)):
    __dir = [__dir for __dir in sorted(os.listdir(os.path.join(real_path, directories[i],_dir[j])))]
    for k in range(len(__dir)):
      __dir[k] = '/'.join((real_path, directories[i], _dir[j],__dir[k]))
    data_dir[i].append(__dir)

In [ ]:
y = []
X_dir = []
for i in range(len(data_dir)):
  for j in range(len(data_dir[i])):
    for k in range(len(data_dir[i][j])):
      X_dir.append(data_dir[i][j][k])
      y.append(j)

In [ ]:
X_dir = np.asarray(X_dir)
y = np.array(y)
shuffle_index = np.random.permutation(20000)
X_dir, y = X_dir[shuffle_index], y[shuffle_index]

X_train_dir, y_train = X_dir[:15000], y[:15000] 
X_test_dir, y_test = X_dir[15000:], y[15000:]

In [ ]:
X_train_dir.shape

In [ ]:
import shutil
os.makedirs('gesture_datasets')

In [ ]:
path = 'gesture_datasets'
for i in X_dir:
  shutil.copy(i,path)

### As the data does not fit in memeory get the file in batches

In [ ]:
def get_batches(index, batch_size):
  X_batch = list()
  y_batch = list()

  first_index = index*batch_size
  final_index = (index+1)*batch_size
  while first_index < final_index:
    image = plt.imread(X_train_dir[first_index])
    image_resized = resize(image, (224,224,3))
    X_batch.append(image_resized)
    y_batch.append(y_train[first_index])
    first_index = first_index + 1; 
    X_batch_final = np.asarray(X_batch)
    y_batch_final = np.array(y_batch)
  return X_batch_final, y_batch_final

#### some files may use later

In [ ]:
X_some, y_some = [],[]
for index in range(len(X_test_dir)):
  X_some.append(resize(plt.imread(X_test_dir[index]), (224,224,3)))
  y_some.append(y_test[index])


X_some = np.asarray(X_some)
y_some = np.array(y_some)


In [ ]:
X_some = np.asarray(X_some)
y_some = np.array(y_some)

### Trying to use vgg16



In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

base_model = tf.keras.applications.VGG16(
    include_top=False, weights="imagenet",
    input_shape=(224,224,3), 
)

for layers in base_model.layers:
  layers.trainable = False

base_model.summary()

preprocess_input = tf.keras.applications.vgg16.preprocess_input
flatten_layer = tf.keras.layers.Flatten()
intermidate_1 = tf.keras.layers.Dense(476, activation='elu', name='fc1' kernel_initializer="he_normal")
intermidate_2 = tf.keras.layers.Dense(324, activation='elu', name='fc2', kernel_initializer="he_normal")
prediction_layer = tf.keras.layers.Dense(10, activation="softmax")
inputs = tf.keras.Input(shape=(224,224,3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x)
x = flatten_layer(x)
x = intermidate_1(x)
x = intermidate_2(x)
output = prediction_layer(x)

model = tf.keras.Model(inputs, output)

SyntaxError: ignored

In [ ]:
from tensorflow.keras import models, layers
new_model=models.Sequential()
new_model.add(layers.Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(224, 224,3))) 
new_model.add(layers.MaxPooling2D((2, 2)))
new_model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
new_model.add(layers.MaxPooling2D((2, 2)))
new_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
new_model.add(layers.MaxPooling2D((2, 2)))
new_model.add(layers.Flatten())
new_model.add(layers.Dropout(0.5))
new_model.add(layers.Dense(128, activation='relu'))
new_model.add(layers.Dropout(0.5))
new_model.add(layers.Dense(10, activation='softmax'))

# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# optim = tf.keras.optimizers.Adam(learning_rate=0.001)
# train_acc_metrics = tf.keras.metrics.SparseCategoricalAccuracy()

# new_model.compile(optimizer=optim, loss=loss_fn, metrics=train_acc_metrics)

# new_model.fit(X_some, y_some, callbacks=[tf.keras.callbacks.EarlyStopping(patience=4)], epochs=8)

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = tf.keras.optimizers.Adam(learning_rate=0.01)
train_acc_metrics = tf.keras.metrics.SparseCategoricalAccuracy()

checkpoint_dir = "/content/drive/MyDrive/tf_ckpts"
#os.mkdir(checkpoint_dir)

ckpt = tf.train.Checkpoint(step=tf.Variable(1), model=new_model, optimizer=optim)
manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=6)

In [ ]:
import time
n_epochs = 3
batch_size = 64

train_loss, train_accuracy = [],[]

ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
else:
    print("Initializing from scratch.")
total_time_start = time.time()
for epoch in range(n_epochs):
  batch_start_time = time.time()
  for iteration in range(15000 // batch_size):
    X_batch_resized = []
    X_batch, y_batch = get_batches(iteration, batch_size)
    X_batch, y_batch = augmentation.flow(X_batch,y_batch, 64)[0]
    # X_batch = np.reshape(X_batch, (-1, 224, 224, 3))
    # X_batch, y_batch = my_training_data_generator[0][0], my_training_data_generator[0][1]
    with tf.GradientTape() as tape:
      logits = new_model(X_batch)
      loss_value = loss_fn(y_batch, logits)
    grads = tape.gradient(loss_value, new_model.trainable_variables)
    optim.apply_gradients(zip(grads, new_model.trainable_variables))

    train_acc_metrics.update_state(y_batch, logits)
    batch_accuracy = train_acc_metrics.result()
    train_accuracy.append(batch_accuracy.numpy())
    train_loss.append(loss_value.numpy())
    print("batch", iteration, "accuracy: ", batch_accuracy.numpy(), "batch", iteration," ----  " "loss", loss_value.numpy())
    if iteration % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (iteration, float(loss_value))
            )
            print("Seen so far: %d samples" % ((iteration + 1) * 10))
            print("time(seconds): ", time.time()-batch_start_time)

  ckpt.step.assign_add(1)
  save_path = manager.save()

  # Display metrics at the end of each epoch.
  train_acc = train_acc_metrics.result()
  print("Training acc over epoch: %.4f" % (float(train_acc),))

  # Reset training metrics at the end of each epoch
  train_acc_metrics.reset_states()

  

  print("Saved checkpoint for epoch {}: {}".format(int(ckpt.step), save_path))

print("Total time taken", total_time_start - time.time())

new_model.save("/content/drive/MyDrive/mode1_3.h5")


In [ ]:
from sklearn.metrics import accuracy_score
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/mode1_3.h5")

y_predict = np.argmax(loaded_model.predict(X_some),axis=1)
score = accuracy_score(y_some, y_predict)

In [ ]:
score

0.9747126436781609

### Using fit_generator with image augmentaion

In [ ]:
augmentation = ImageDataGenerator(
  rotation_range=20,
  zoom_range=0.15,
	width_shift_range=0.2,
	horizontal_flip=True)

In [ ]:
class My_Custom_Generator(tf.keras.utils.Sequence):
  def __init__(self, X_dir, y, batch_size):
    self.X_dir = X_dir
    self.y = y
    self.batch_size = batch_size
  def __len__(self):
      return (np.ceil(len(self.X_dir) / float(self.batch_size))).astype(np.int)
  def __getitem__(self, idx) :
    batch_x = self.X_dir[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.y[idx * self.batch_size : (idx+1) * self.batch_size]
    
    images,labels = np.array([
            resize(plt.imread(file_name), (224, 224, 3))
               for file_name in batch_x]),np.array(batch_y)

    # (images, labels) = next(augmentation.flow(images, labels, 64))
    return images, labels


In [ ]:
my_training_data_generator = My_Custom_Generator(X_dir[:15000], y[:15000], 64)
my_validation_data_generator = My_Custom_Generator(X_dir[15000:], y[15000:], 64)

In [ ]:
image = my_training_data_generator[0][0][1]
label = my_training_data_generator[0][1][1]
plt.imshow(image)
plt.show()

In [ ]:
new_model.compile(optimizer=optim, loss=loss_fn, metrics=train_acc_metrics)

In [ ]:
history = new_model.fit(x=My_Custom_Generator(X_dir[:15000], y[:15000], 64),
              steps_per_epoch= int(15000 // 64),
              epochs = 30,
              callbacks=tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/mode1_5.h5')
              )
new_model.save("/content/drive/MyDrive/mode1_5.h5")

In [ ]:
classes = ['palm', 'I', 'fist', 'fist_moved', 'thumb', 'index', 'ok', 'palm_moved', 'c', 'down']